# Problem 4 : Estimation - BLP

In [1]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra,StatsFuns

Random.seed!(6789998212);

In [2]:
# load in csv
df = DataFrame(CSV.File("../data/ps1_ex4.csv"));

In [3]:
# simulate individual taste shocks from N(μ,Σ)
draw_sim = function(μ, Σ, N) # return N x L matrix
    # draw shocks
    v = rand(MvNormal(μ, Σ), N)
    
    return v
end

#1 (generic function with 1 method)

In [29]:
# get data
n_markets = 10
n_sim = 50

x_t = []
for t in 1:n_markets 
   push!(x_t, Array(df[df[!,:market].==t, [:p, :x]]))
end

s_t = []
for t in 1:n_markets 
    push!(s_t, Array(df[df[!,:market].==t, [:shares]]))
end

z_t = []
for t in 1:n_markets 
    push!(z_t, Array(df[df[!,:market].==t, [:z1, :z2, :z3, :z4, :z5, :z6, :x]]))
end

x_jt = Array(df[df[!,:market] .<= n_markets,[:p, :x]]);

z_jt = Array(df[df[!,:market] .<= n_markets,[:z1, :z2, :z3, :z4, :z5, :z6, :x]]);

v = draw_sim([0;0], [1 0;0 1], n_sim)

2×50 Matrix{Float64}:
 -0.403395  0.00531162  -0.106624   …   1.12778  -0.902155  -0.133108
 -0.62395   0.185082    -0.0156497     -2.54691  -0.226052  -0.613839

# Part 1: BLP

## Inner loop
`get_shares` calculates the shares of each product in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a matrix of size $J \times 2$; and $v$ should be a vector of length $L$.

`delta_contraction` iterates the $\delta_{jt}$ in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a vector of characteristics with length $J$; $s$ should be a vector of observed shares with length $J$; $v$ should be a vector of length $L$. 

`market_iterate` performs the contraction over each $t$ markets, it recoves $\delta_{jt}$, which is a vector of length $J \times T$.

In [18]:
# get shares in a market given some fixed gamma and delta
get_shares = function(δ, Γ, x, v)
    # we want to get share_{jt} using simulated values of $v_i$ (drawn above)
    # shares should be vector of length J
    numerator = exp.(δ .+ x * Γ * v)
    adj = maximum(numerator, dims = 1)
    denominator = sum((numerator ./ adj), dims = 1) .+ (1 ./ adj)
    shares = sum((numerator ./ adj) ./ denominator, dims = 2) ./ size(v)[2]
    
    return shares
end

# inner loop: contraction to find δ
delta_contraction = function(δ₀, Γ, s, x, v, tol = 1e-12, max_iter = nothing)

    # here δ is a vector of length J
    δ = δ₀
    err = 1000
    n = 0
    maxed_iter = false
    
    while (err > tol) && (maxed_iter === false)
        δ_old = δ
        
        # update delta
        δ = δ_old + log.(s) - log.(get_shares(δ_old, Γ, x, v))
        
        # difference 
        err = maximum(abs.(δ - δ_old)) 
        
        # (optional) max iterations block
        n += 1
        if max_iter !== nothing
            maxed_iter = (n == max_iter)
        end
    end
    
    return δ
end

# iterate over each market
market_iterate = function(Γ, s_t, x_t, v, tol = 1e-12, max_iter = nothing)
   
    δ = []
    for t in 1:size(s_t)[1]
        s = s_t[t]
        x = x_t[t]
        δ₀ = ones(size(s)[1])
        push!(δ, delta_contraction(δ₀, Γ, s, x, v, tol, max_iter) ) 
    end
    return δ
end

#31 (generic function with 3 methods)

## Outer loop
`residuals` does IV-GMM using the provided weighting matrix. z_jt should be a matrix of $Z$ excluded and included intruments of size $TJ \times Z$. Returns linear parameters (vector of length $2$) and $\xi_{jt}$ residuals (vector of length $J \times T$)

`gmm_objective` Reads in $TJ$-length vector $x$_jt and $TJ \times Z$ matrix $z$_jt. Calculates sample moments (size of instrument vector, $Z$) and optimal weighting matrix ($Z \times Z$). Returns scalar objective and matrix.

## Gradient
Estimator is $$\nabla G(\theta) = 2(Z'J_\theta)'W(Z'\xi(\theta))$$
-$Z$ is $JT \times Z$ matrix if instruments

-$\xi$ is $JT \times 1$ matrix of unobserved mean utilities

-$W$ is $Z \times Z$ 

$$ W = \left[(\xi(\theta) \circ Z)' (\xi(\theta) \circ Z) \right]^{-1}$$

-$J_\theta$ is $JT \times 3$

$$ J_\theta = -f_\xi^{-1} f_\theta$$

- For each t, $f_\xi$ is a $J \times J$ matrix: $\left\{\frac{\partial s_{ij}}{\partial \xi_k}\right\}_{j,k}$

$$ \frac{\partial s_{ij}}{\partial \xi_k} = -s_{ij}s_{ik}, \quad \frac{\partial s_{ij}}{\partial \xi_j} = s_{ij}(1-s_{ij}) $$

- For each t, $f_\theta$ is a $J \times 3$ matrix: 
$$\begin{bmatrix} s_{ij}\left(p_j - \sum_k s_{ik}p_k \right)\nu_{1i}  &  s_{ij}\left(x_j - \sum_k s_{ik}x_k \right)\nu_{1i} & s_{ij}\left(x_j - \sum_k s_{ik}x_k \right)\nu_{2i} \end{bmatrix}_{j} $$

All matrices are stacked $J \times \cdot$ over $T$ markets

In [37]:
# returns residuals for a given δ, estimates linear parameters given instruments
resid = function(δ_jt, x_jt, z_jt, W)
    # iv-gmm
    θ₁ = inv(x_jt' * z_jt * W * z_jt' * x_jt) * (x_jt' * z_jt * W * z_jt' * δ_jt)
    ξ_jt = δ_jt - x_jt * θ₁
    
    return ξ_jt, θ₁ 
    
end

# calculates gmm objective for outer loop
function gmm_objective(ξ_jt, z_jt, W)   
    # empirical moments, weighting matrix
    g = (ξ_jt' * z_jt) / size(ξ_jt)[1] 
    
    # gmm objective
    G = g * W * g'
    
    return G
end

# performs outer loop
function outer_loop(θ₂, s_t, x_t, x_jt, z_jt, v, W, tol = 1e-12, max_iter = nothing)
    # Pass through guess
    Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]] # lower triangular
    
    # Perform inner loop
    δ = market_iterate(Γ, s_t, x_t, v, tol, max_iter)
    
    # convert to JT x 1 (stacked J x 1 vectors for each t)
    δ_jt = vec(reduce(hcat,δ)) 
    
    # intermediate step
    ξ_jt, θ₁ = resid(δ_jt, x_jt, z_jt, W)
    
    # gmm step
    G = gmm_objective(ξ_jt, z_jt, W)
    
    return G
end

# Steps to calculate gradient...
# have data s_t, x_t, z_t
# first step will return θ           -> Γ  x
# 1. run market_iterate() with Γ     -> δ  x
# 2. run resid() with δ              -> ξ x
# 3. calculate W with ξ and Z        -> W x
# 4. calculate J with δ and Γ*       -> J  x
# *(for each i, calculate s_ij vector, do elementwise mult with p_j, v, and sum to get f_xi loop through j,k for f_theta)
# 5. run gradient() with J, W, ξ, Z  -> ∇

# helper function in gradient call: for each market get Jacobian of ξ(θ)
function jacobian_xi(δ, Γ, x, v)
    # need individual shares
    numerator = exp.(δ .+ x * Γ * v)
    adj = maximum(numerator, dims = 1)
    denominator = sum((numerator ./ adj), dims = 1) .+ (1 ./ adj)
    shares = (numerator ./ adj) ./ denominator # J x L
    
    # calculate partials of f(θ) = s - S(ξ,θ), denoted fξ and fθ
    fξ_store = []
    fθ_store = [] 
    for i = 1:size(v)[2]
        s_i = shares[:,i]
        v_i = v[:,i]
        
        fξ_i = s_i * s_i' + diagm(s_i)
        
        fθ₁ = s_i .* (x[:,1] .- (s_i' * x[:,1])) .* v[1]
        fθ₂ = s_i .* (x[:,2] .- (s_i' * x[:,2])) .* v[1]
        fθ₃ = s_i .* (x[:,2] .- (s_i' * x[:,2])) .* v[2]
        fθ_i = hcat(fθ₁, fθ₂, fθ₃)
        
        push!(fξ_store, fξ_i)
        push!(fθ_store, fθ_i)        
    end
    
    # calculate Jacobian
    J = inv(mean(fξ_store)) * mean(fθ_store)
    
    return J
end
    
function gmm_gradient!(θ₂, s_t, x_t, x_jt, z_jt, v, W, ∇, tol = 1e-12, max_iter = nothing)
    # Pass through guess
    Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]] # lower triangular
    
    # Recover model objects from estimates parameters: Γ, and data: s_t, x_t, z_t, and v (simulated)    
    # δ(θ)
    δ = market_iterate(Γ, s_t, x_t, v, tol, max_iter)
    
    # ξ(θ)
    δ_jt = vec(reduce(hcat,δ)) 
    ξ_jt = resid(δ_jt, x_jt, z_jt, W)[1]
    
    # Analytic matrices
    # Jacobian
    J_t = []
    for t = 1:size(x_t)[1]
        push!(J_t, jacobian_xi(δ[t], Γ, x_t[t], v))
    end
    J = reduce(vcat, J_t) # flatten to JT x 3 matrix
    
    # Weighting (note: put outside, we want to fix W through run)
    # W = inv((z_jt .* ξ_jt)' * (z_jt .* ξ_jt)) * size(ξ_jt)[1]
    
    # Calculate gradient
    ∇ = 2 .* (z_jt' * J)' * W * (z_jt' * ξ_jt)
    
    return ∇
end

gmm_gradient! (generic function with 3 methods)

In [41]:
δ₀ = ones(6)
params0 = ones(3)
Γ = [params0[1] 0 ; params0[2] params0[3]]

w = inv(z_jt' * z_jt) * size(z_jt)[1];

tol = 1e-16;
max_iter = nothing;

f(θ₂) = outer_loop(θ₂, s_t, x_t, x_jt, z_jt, v, w, tol, max_iter)
g!(∇,θ₂) = gmm_gradient!(θ₂, s_t, x_t, x_jt, z_jt, v, w, ∇, tol, max_iter)

g! (generic function with 1 method)

In [ ]:
@time o = Optim.optimize(f, g!,params0, LBFGS())

In [124]:
J_t = []
for t = 1:5
    push!(J_t, jacobian(delt[1], Γ, x_t[1], v))
end

J_t[1]
reduce(vcat, J_t)

30×3 Matrix{Float64}:
 -0.0538056    0.0193898    0.176645
 -0.0103725   -0.0587022   -0.53479
  0.0692869   -0.0495491   -0.451403
 -0.00318613  -0.0494922   -0.450885
  0.0317762    0.022883     0.208469
  0.0688848    0.00671953   0.0612165
 -0.0538056    0.0193898    0.176645
 -0.0103725   -0.0587022   -0.53479
  0.0692869   -0.0495491   -0.451403
 -0.00318613  -0.0494922   -0.450885
  0.0317762    0.022883     0.208469
  0.0688848    0.00671953   0.0612165
 -0.0538056    0.0193898    0.176645
  ⋮                        
 -0.0538056    0.0193898    0.176645
 -0.0103725   -0.0587022   -0.53479
  0.0692869   -0.0495491   -0.451403
 -0.00318613  -0.0494922   -0.450885
  0.0317762    0.022883     0.208469
  0.0688848    0.00671953   0.0612165
 -0.0538056    0.0193898    0.176645
 -0.0103725   -0.0587022   -0.53479
  0.0692869   -0.0495491   -0.451403
 -0.00318613  -0.0494922   -0.450885
  0.0317762    0.022883     0.208469
  0.0688848    0.00671953   0.0612165

In [104]:
δ_jt = vec(reduce(hcat, delt))
z_jt

600×7 Matrix{Float64}:
 -6.36336    -5.50945   -5.37102   …  -1.93707   -0.34181    -0.210973
 -5.19816    -4.39604   -4.26527      -3.49809   -2.33013    -1.55758
 -0.0408355   0.835758  -0.520122      0.768238  -0.973296   -0.799984
 -1.53891    -1.1727     0.189448      1.51942   -0.264684   -1.33837
  0.758555   -0.70047    0.723864     -1.28846    0.0804491   3.31633
  1.04464    -0.724283  -1.32533   …  -0.922602   1.71668     2.15453
 -5.87798    -4.01759   -5.99733      -1.8697     0.298796    0.470776
 -6.24899    -4.75995   -5.71777      -3.64392   -1.61345    -0.153134
  0.851501    0.426509   1.73536      -1.09451   -0.576743   -2.18476
  1.74984     2.34954   -0.404193     -0.860983   0.153472   -0.743471
  0.0580351  -1.19704    1.10368   …   0.265751  -1.58688     1.56558
 -0.181291   -0.372407   0.763489      0.520254   0.125024    3.14475
 -6.18583    -4.48453   -5.34863      -2.46034   -0.186484   -0.0839182
  ⋮                                ⋱              ⋮         

In [115]:
w = z_jt' * z_jt
ξ = resid(δ_jt, x_jt, z_jt, w)[1]

inv((z_jt .* ξ)' * (z_jt .* ξ)) * size(ξ)[1]


7×7 Matrix{Float64}:
  0.00417211   -0.00115626   -0.00174725   …  -5.87426e-5   -0.00128971
 -0.00115626    0.00446283   -0.00199852      -0.000183901   0.000156218
 -0.00174725   -0.00199852    0.00438948      -0.000482567   0.000395504
 -0.00197874    0.000243024   0.000267834      3.56317e-6    0.000798649
 -0.000504168  -0.000780754  -0.000550856      0.000684392   3.69086e-5
 -5.87426e-5   -0.000183901  -0.000482567  …   0.00689566    0.000495682
 -0.00128971    0.000156218   0.000395504      0.000495682   0.00130443

In [99]:
g, w = outer_loop([1 1 1], s_t, x_t, x_jt, z_jt, v)

(0.21896565080926234, [0.085603451180656 -0.034882942297747 … -0.005707878298005312 -0.005566964348651631; -0.03488294229774701 0.08969824619390354 … 0.007811092682178221 -0.005534110983618899; … ; -0.005707878298005316 0.007811092682178222 … 0.12877094892746987 0.002128356730109853; -0.005566964348651642 -0.0055341109836188895 … 0.0021283567301098524 0.09548174494899414])

In [48]:
delt = market_iterate(Γ, s_t, x_t, v, 1e-1, 3)
delt[1]
get_shares(delt[1], Γ, x_t[1], v)[1]

[0.000277852 -0.210972888; 0.000457065 -1.557577227; 1.016838207 -0.799983823; 0.109472238 -1.338366198; 5.387785919 3.316330263; 4.188118546 2.154526537]
[1.0 0.0; 1.0 1.0]
[0.12356101813854255 -0.2011264856996698 0.7806282938685422 -0.09844542892614082 -0.413408443148965 1.5968214742984805 -1.0790156916848874 -2.3054904164006382 0.8200018913655547 0.03928676588091386 0.4721192659326684 -0.678373609941218 -0.7978657312983021 0.15850689883790126 -0.45535890190946465 0.39248124375555454 0.5345804565960858 1.1917200259581169 -1.088560988767867 -0.5446622311712501 0.03474397067085089 -2.4769455055591476 -0.6075241254336914 2.033737198025753 -0.380103268845033 -1.1866271219729219 -1.254892287280896 0.2608198163458342 1.7763180149557478 -1.7338939246748841 0.5587937348101978 -0.445094847234231 0.5607960291545503 1.7684373338503636 0.13096485624723075 -1.4467557444937271 0.12432686222877065 -0.8348861019653572 -0.2770858473130447 0.22832508896392256 1.9124621940564983 0.38669071938121957 -0.

[0.12356101813854255 -0.2011264856996698 0.7806282938685422 -0.09844542892614082 -0.413408443148965 1.5968214742984805 -1.0790156916848874 -2.3054904164006382 0.8200018913655547 0.03928676588091386 0.4721192659326684 -0.678373609941218 -0.7978657312983021 0.15850689883790126 -0.45535890190946465 0.39248124375555454 0.5345804565960858 1.1917200259581169 -1.088560988767867 -0.5446622311712501 0.03474397067085089 -2.4769455055591476 -0.6075241254336914 2.033737198025753 -0.380103268845033 -1.1866271219729219 -1.254892287280896 0.2608198163458342 1.7763180149557478 -1.7338939246748841 0.5587937348101978 -0.445094847234231 0.5607960291545503 1.7684373338503636 0.13096485624723075 -1.4467557444937271 0.12432686222877065 -0.8348861019653572 -0.2770858473130447 0.22832508896392256 1.9124621940564983 0.38669071938121957 -0.8948803787796222 -0.041757831206128714 -0.9800838709510995 0.7284409490591465 0.8939435620019291 -1.4165690363601433 -0.895813735719662 -2.118604414166602; 0.8050050767019042

[0.12356101813854255 -0.2011264856996698 0.7806282938685422 -0.09844542892614082 -0.413408443148965 1.5968214742984805 -1.0790156916848874 -2.3054904164006382 0.8200018913655547 0.03928676588091386 0.4721192659326684 -0.678373609941218 -0.7978657312983021 0.15850689883790126 -0.45535890190946465 0.39248124375555454 0.5345804565960858 1.1917200259581169 -1.088560988767867 -0.5446622311712501 0.03474397067085089 -2.4769455055591476 -0.6075241254336914 2.033737198025753 -0.380103268845033 -1.1866271219729219 -1.254892287280896 0.2608198163458342 1.7763180149557478 -1.7338939246748841 0.5587937348101978 -0.445094847234231 0.5607960291545503 1.7684373338503636 0.13096485624723075 -1.4467557444937271 0.12432686222877065 -0.8348861019653572 -0.2770858473130447 0.22832508896392256 1.9124621940564983 0.38669071938121957 -0.8948803787796222 -0.041757831206128714 -0.9800838709510995 0.7284409490591465 0.8939435620019291 -1.4165690363601433 -0.895813735719662 -2.118604414166602; 0.8050050767019042

[0.12356101813854255 -0.2011264856996698 0.7806282938685422 -0.09844542892614082 -0.413408443148965 1.5968214742984805 -1.0790156916848874 -2.3054904164006382 0.8200018913655547 0.03928676588091386 0.4721192659326684 -0.678373609941218 -0.7978657312983021 0.15850689883790126 -0.45535890190946465 0.39248124375555454 0.5345804565960858 1.1917200259581169 -1.088560988767867 -0.5446622311712501 0.03474397067085089 -2.4769455055591476 -0.6075241254336914 2.033737198025753 -0.380103268845033 -1.1866271219729219 -1.254892287280896 0.2608198163458342 1.7763180149557478 -1.7338939246748841 0.5587937348101978 -0.445094847234231 0.5607960291545503 1.7684373338503636 0.13096485624723075 -1.4467557444937271 0.12432686222877065 -0.8348861019653572 -0.2770858473130447 0.22832508896392256 1.9124621940564983 0.38669071938121957 -0.8948803787796222 -0.041757831206128714 -0.9800838709510995 0.7284409490591465 0.8939435620019291 -1.4165690363601433 -0.895813735719662 -2.118604414166602; 0.8050050767019042

[0.12356101813854255 -0.2011264856996698 0.7806282938685422 -0.09844542892614082 -0.413408443148965 1.5968214742984805 -1.0790156916848874 -2.3054904164006382 0.8200018913655547 0.03928676588091386 0.4721192659326684 -0.678373609941218 -0.7978657312983021 0.15850689883790126 -0.45535890190946465 0.39248124375555454 0.5345804565960858 1.1917200259581169 -1.088560988767867 -0.5446622311712501 0.03474397067085089 -2.4769455055591476 -0.6075241254336914 2.033737198025753 -0.380103268845033 -1.1866271219729219 -1.254892287280896 0.2608198163458342 1.7763180149557478 -1.7338939246748841 0.5587937348101978 -0.445094847234231 0.5607960291545503 1.7684373338503636 0.13096485624723075 -1.4467557444937271 0.12432686222877065 -0.8348861019653572 -0.2770858473130447 0.22832508896392256 1.9124621940564983 0.38669071938121957 -0.8948803787796222 -0.041757831206128714 -0.9800838709510995 0.7284409490591465 0.8939435620019291 -1.4165690363601433 -0.895813735719662 -2.118604414166602; 0.8050050767019042

...

[1.0 0.0; 1.0 1.0]
[0.12356101813854255 -0.2011264856996698 0.7806282938685422 -0.09844542892614082 -0.413408443148965 1.5968214742984805 -1.0790156916848874 -2.3054904164006382 0.8200018913655547 0.03928676588091386 0.4721192659326684 -0.678373609941218 -0.7978657312983021 0.15850689883790126 -0.45535890190946465 0.39248124375555454 0.5345804565960858 1.1917200259581169 -1.088560988767867 -0.5446622311712501 0.03474397067085089 -2.4769455055591476 -0.6075241254336914 2.033737198025753 -0.380103268845033 -1.1866271219729219 -1.254892287280896 0.2608198163458342 1.7763180149557478 -1.7338939246748841 0.5587937348101978 -0.445094847234231 0.5607960291545503 1.7684373338503636 0.13096485624723075 -1.4467557444937271 0.12432686222877065 -0.8348861019653572 -0.2770858473130447 0.22832508896392256 1.9124621940564983 0.38669071938121957 -0.8948803787796222 -0.041757831206128714 -0.9800838709510995 0.7284409490591465 0.8939435620019291 -1.4165690363601433 -0.895813735719662 -2.118604414166602;

[0.12356101813854255 -0.2011264856996698 0.7806282938685422 -0.09844542892614082 -0.413408443148965 1.5968214742984805 -1.0790156916848874 -2.3054904164006382 0.8200018913655547 0.03928676588091386 0.4721192659326684 -0.678373609941218 -0.7978657312983021 0.15850689883790126 -0.45535890190946465 0.39248124375555454 0.5345804565960858 1.1917200259581169 -1.088560988767867 -0.5446622311712501 0.03474397067085089 -2.4769455055591476 -0.6075241254336914 2.033737198025753 -0.380103268845033 -1.1866271219729219 -1.254892287280896 0.2608198163458342 1.7763180149557478 -1.7338939246748841 0.5587937348101978 -0.445094847234231 0.5607960291545503 1.7684373338503636 0.13096485624723075 -1.4467557444937271 0.12432686222877065 -0.8348861019653572 -0.2770858473130447 0.22832508896392256 1.9124621940564983 0.38669071938121957 -0.8948803787796222 -0.041757831206128714 -0.9800838709510995 0.7284409490591465 0.8939435620019291 -1.4165690363601433 -0.895813735719662 -2.118604414166602; 0.8050050767019042

[0.12356101813854255 -0.2011264856996698 0.7806282938685422 -0.09844542892614082 -0.413408443148965 1.5968214742984805 -1.0790156916848874 -2.3054904164006382 0.8200018913655547 0.03928676588091386 0.4721192659326684 -0.678373609941218 -0.7978657312983021 0.15850689883790126 -0.45535890190946465 0.39248124375555454 0.5345804565960858 1.1917200259581169 -1.088560988767867 -0.5446622311712501 0.03474397067085089 -2.4769455055591476 -0.6075241254336914 2.033737198025753 -0.380103268845033 -1.1866271219729219 -1.254892287280896 0.2608198163458342 1.7763180149557478 -1.7338939246748841 0.5587937348101978 -0.445094847234231 0.5607960291545503 1.7684373338503636 0.13096485624723075 -1.4467557444937271 0.12432686222877065 -0.8348861019653572 -0.2770858473130447 0.22832508896392256 1.9124621940564983 0.38669071938121957 -0.8948803787796222 -0.041757831206128714 -0.9800838709510995 0.7284409490591465 0.8939435620019291 -1.4165690363601433 -0.895813735719662 -2.118604414166602; 0.8050050767019042

[0.12356101813854255 -0.2011264856996698 0.7806282938685422 -0.09844542892614082 -0.413408443148965 1.5968214742984805 -1.0790156916848874 -2.3054904164006382 0.8200018913655547 0.03928676588091386 0.4721192659326684 -0.678373609941218 -0.7978657312983021 0.15850689883790126 -0.45535890190946465 0.39248124375555454 0.5345804565960858 1.1917200259581169 -1.088560988767867 -0.5446622311712501 0.03474397067085089 -2.4769455055591476 -0.6075241254336914 2.033737198025753 -0.380103268845033 -1.1866271219729219 -1.254892287280896 0.2608198163458342 1.7763180149557478 -1.7338939246748841 0.5587937348101978 -0.445094847234231 0.5607960291545503 1.7684373338503636 0.13096485624723075 -1.4467557444937271 0.12432686222877065 -0.8348861019653572 -0.2770858473130447 0.22832508896392256 1.9124621940564983 0.38669071938121957 -0.8948803787796222 -0.041757831206128714 -0.9800838709510995 0.7284409490591465 0.8939435620019291 -1.4165690363601433 -0.895813735719662 -2.118604414166602; 0.8050050767019042

LoadError: UndefVarError: v not defined

In [ ]:
θ₂ = o.minimizer
Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]]

δ = market_iterate(Γ, s_t, x_t, v)
    
δ_jt = vec(reduce(hcat,δ)) 
ξ_jt, θ₁ = resid(δ_jt, x_jt, z_jt, w)
    
S_t = []
for t = 1:size(x_t)[1]
    push!(S_t, get_shares(δ[t], Γ, x_t[t], v))
end
S = mean(reduce(hcat,S_t), dims = 2)

s = mean(reduce(hcat,s_t), dims = 2)

print(\theta\_1)
maximum(reduce(hcat,S_t) - reduce(hcat,s_t))